In [1]:
!pwd

/Users/alayah361/codes/fluctmatch/notebooks


In [2]:
from os import path
from shutil import copyfile
from fluctmatch import avg_dcd_noh

rootfolder = '/Users/alayah361/fluctmatch'
na_mechfolder = '/Users/alayah361/codes/fluctmatch'
# na_mechfolder = '/home/yizaochen/codes/na_mechanics'
d_single_na = {'arna+arna': 'arna', 'bdna+bdna': 'bdna'}
vmd = '/usr/local/bin/vmd'

### Step 1: Initialize

In [3]:
host = 'nome'
type_na = 'bdna+bdna'
n_bp = 13
begin_frame = 1
frame_num = 10000

agent = avg_dcd_noh.AvgcrddcdAgent(host, type_na, rootfolder)

/Users/alayah361/fluctmatch/nome exists
/Users/alayah361/fluctmatch/nome/bdna+bdna exists
/Users/alayah361/fluctmatch/nome/bdna+bdna/input exists
/Users/alayah361/fluctmatch/nome/bdna+bdna/input/allatoms exists
/Users/alayah361/fluctmatch/nome/bdna+bdna/input/heavyatoms exists
/Users/alayah361/fluctmatch/nome/bdna+bdna/charmm_inp exists
/Users/alayah361/fluctmatch/nome/bdna+bdna/charmm_dat exists
/Users/alayah361/fluctmatch/nome/bdna+bdna/make_crd exists


### Step 2: Prepare dcd and pdb

In [15]:
inish = path.join(na_mechfolder, 'shell_scripts', 'initialize_input.sh')
print(f'cp {host}.pdb {host}.xtc {rootfolder}/{host}/{type_na}/allatoms')
print(f'vi /Users/alayah361/codes/fluctmatch/shell_scripts/initialize_input.sh\n(remember to change:\n{rootfolder}\n{host}\n{type_na})\n')
cmd = f'bash {inish}'
print(cmd)
cmd = f'r 1-{n_bp}'
print(cmd)

cp nome.pdb nome.xtc /Users/alayah361/fluctmatch/nome/bdna+bdna/allatoms
vi /Users/alayah361/codes/fluctmatch/shell_scripts/initialize_input.sh
(remember to change:
/Users/alayah361/fluctmatch
nome
bdna+bdna)

bash /Users/alayah361/codes/fluctmatch/shell_scripts/initialize_input.sh
r 1-13


In [16]:
# Manual Delete
aafolder = agent.aa_folder
single_na = d_single_na[type_na]
temp_pdb = path.join(aafolder, f'{single_na}1.central.pdb')
print('manually delet the oppsite strand:')
cmd = f'vim {temp_pdb}'
print(cmd)
temp_pdb = path.join(aafolder, f'{single_na}2.central.pdb')
cmd = f'vim {temp_pdb}'
print(cmd)

manually delet the oppsite strand:
vim /Users/alayah361/fluctmatch/nome/bdna+bdna/input/allatoms/bdna1.central.pdb
vim /Users/alayah361/fluctmatch/nome/bdna+bdna/input/allatoms/bdna2.central.pdb


In [19]:
pdbcharmmsh = path.join(na_mechfolder, 'shell_scripts', 'pdb_gro2charmm.sh')
print(f'vi /Users/alayah361/codes/fluctmatch/shell_scripts/pdb_gro2charmm.sh\n(remember to change:\n{rootfolder}\n{host} {type_na} 1 or 2)\n')

cmd = f'bash {pdbcharmmsh} {rootfolder} {host} {type_na} 1'
print(cmd)
cmd = f'bash {pdbcharmmsh} {rootfolder} {host} {type_na} 2'
print(cmd)

vi /Users/alayah361/codes/fluctmatch/shell_scripts/pdb_gro2charmm.sh
(remember to change:
/Users/alayah361/fluctmatch
nome bdna+bdna 1 or 2)

bash /Users/alayah361/codes/fluctmatch/shell_scripts/pdb_gro2charmm.sh /Users/alayah361/fluctmatch nome bdna+bdna 1
bash /Users/alayah361/codes/fluctmatch/shell_scripts/pdb_gro2charmm.sh /Users/alayah361/fluctmatch nome bdna+bdna 2


In [20]:
temp_pdb = path.join(agent.aa_folder, f'{type_na}.central.pdb')
temp_xtc = path.join(agent.aa_folder, f'{type_na}.central.xtc')
temp_dcd = path.join(agent.aa_folder, f'{type_na}.central.dcd')
print('back to bash and open vmd:')
cmd = f'vmd -pdb {temp_pdb} -xtc {temp_xtc}'
print(cmd)
cmd = f'\nTK console:\nanimate write dcd {temp_dcd} beg 1 end 10001 waitfor all'
print(cmd)
cmd = f'\nvmd -pdb {temp_pdb} -dcd {temp_dcd}'
print(cmd)

back to bash and open vmd:
vmd -pdb /Users/alayah361/fluctmatch/nome/bdna+bdna/input/allatoms/bdna+bdna.central.pdb -xtc /Users/alayah361/fluctmatch/nome/bdna+bdna/input/allatoms/bdna+bdna.central.xtc

TK console:
animate write dcd /Users/alayah361/fluctmatch/nome/bdna+bdna/input/allatoms/bdna+bdna.central.dcd beg 1 end 10001 waitfor all

vmd -pdb /Users/alayah361/fluctmatch/nome/bdna+bdna/input/allatoms/bdna+bdna.central.pdb -dcd /Users/alayah361/fluctmatch/nome/bdna+bdna/input/allatoms/bdna+bdna.central.dcd


### Step 3: Make CRD (split two strands, then combine) 

##### 創建inp file

In [21]:
#agent.make_crd_input(amber=True, firstter='amber_5ter', lastter='amber_3ter')
agent.make_crd_input(amber=False, firstter='5ter', lastter='3ter')

##### 使用chramm建系統

In [22]:
agent.make_crd()

charmm< /Users/alayah361/fluctmatch/nome/bdna+bdna/make_crd/bdna1.inp > /Users/alayah361/fluctmatch/nome/bdna+bdna/make_crd/bdna1.dat
charmm< /Users/alayah361/fluctmatch/nome/bdna+bdna/make_crd/bdna2.inp > /Users/alayah361/fluctmatch/nome/bdna+bdna/make_crd/bdna2.dat
charmm< /Users/alayah361/fluctmatch/nome/bdna+bdna/make_crd/bdna+bdna.inp > /Users/alayah361/fluctmatch/nome/bdna+bdna/make_crd/bdna+bdna.dat


In [23]:
# # Reset Resid for bdna2.1.pdb, if need
# execute = False
# if execute:
#     offset = -13
#     agent.reset_na2_pdb_resid(offset)

### Step 4: Make CRD and DCD without hydrogen atoms

In [24]:
agent.make_no_h_crd_input(amber = False, firstter = '5ter', lastter = '3ter')

In [25]:
agent.make_no_h_crd()

charmm< /Users/alayah361/fluctmatch/nome/bdna+bdna/charmm_inp/write_no_h_crd.inp > /Users/alayah361/fluctmatch/nome/bdna+bdna/charmm_dat/write_no_h_crd.dat


In [4]:
agent.make_no_h_dcd_input(amber=False, begin=begin_frame, frame_num=frame_num, firstter='5ter', lastter = '3ter')

In [5]:
agent.make_no_h_dcd()

charmm< /Users/alayah361/fluctmatch/nome/bdna+bdna/charmm_inp/write_no_h_dcd.inp > /Users/alayah361/fluctmatch/nome/bdna+bdna/charmm_dat/write_no_h_dcd.dat


### Step 5: Make Average CRD and fitting no-H dcd to average crd

In [28]:
agent.make_avg_crd_input(amber = False, firstter = '5ter', lastter = '3ter')

In [29]:
agent.make_avg_crd()

charmm< /Users/alayah361/fluctmatch/nome/bdna+bdna/charmm_inp/write_no_h_avg_crd.inp > /Users/alayah361/fluctmatch/nome/bdna+bdna/charmm_dat/write_no_h_avg_crd.dat


In [30]:
agent.fit_dcd_to_avg_input(amber = False, begin=begin_frame, frame_num=frame_num, firstter = '5ter', lastter = '3ter')

In [31]:
agent.fit_dcd_to_avg()

charmm< /Users/alayah361/fluctmatch/nome/bdna+bdna/charmm_inp/fit_dcd_to_avg.inp > /Users/alayah361/fluctmatch/nome/bdna+bdna/charmm_dat/fit_dcd_to_avg.dat


### Step 6: Check By VMD

In [32]:
crd = path.join(agent.heavy_folder, f'{type_na}.nohydrogen.avg.crd')
dcd = path.join(agent.heavy_folder, f'{type_na}.nohydrogen.fitavg.dcd')
print(f'vmd -cor {crd} {dcd}')

vmd -cor /Users/alayah361/fluctmatch/nome/bdna+bdna/input/heavyatoms/bdna+bdna.nohydrogen.avg.crd /Users/alayah361/fluctmatch/nome/bdna+bdna/input/heavyatoms/bdna+bdna.nohydrogen.fitavg.dcd


### Additional Part: Copy requried files to allsystem folder

In [58]:
# Copy From simulation folder
simu_folder = '/home/yizaochen/simulation'
simu_datafolder = path.join(simu_folder, host, type_na, 'data')
inputfolder = path.join(rootfolder, host, type_na, 'input', 'allatoms')

old_f = path.join(simu_datafolder, 'gro', f'{type_na}.npt4.fit.gro')
new_f = path.join(inputfolder, f'{type_na}.npt4.all.gro')
copyfile(old_f, new_f)
print(f'cp {old_f} {new_f}')

old_f = path.join(simu_datafolder, 'roughtrj', '1000', f'{type_na}.nopbc.fit.1to10.1000.xtc')
new_f = path.join(inputfolder, f'{type_na}.all.xtc')
copyfile(old_f, new_f)
print(f'cp {old_f} {new_f}')

old_f = path.join(simu_folder, host, type_na, f'{type_na}.gro')
new_f = path.join(inputfolder, f'{type_na}.perfect.gro')
copyfile(old_f, new_f)
print(f'cp {old_f} {new_f}')

cp /home/yizaochen/simulation/gcgc_21mer/bdna+bdna/data/gro/bdna+bdna.npt4.fit.gro /home/yizaochen/codes/dna_rna/all_systems/gcgc_21mer/bdna+bdna/input/allatoms/bdna+bdna.npt4.all.gro
cp /home/yizaochen/simulation/gcgc_21mer/bdna+bdna/data/roughtrj/1000/bdna+bdna.nopbc.fit.1to10.1000.xtc /home/yizaochen/codes/dna_rna/all_systems/gcgc_21mer/bdna+bdna/input/allatoms/bdna+bdna.all.xtc
cp /home/yizaochen/simulation/gcgc_21mer/bdna+bdna/bdna+bdna.gro /home/yizaochen/codes/dna_rna/all_systems/gcgc_21mer/bdna+bdna/input/allatoms/bdna+bdna.perfect.gro


### Reload Function

In [15]:
from imp import reload
reload(avg_dcd_noh)

<module 'avg_dcd_noh' from '/home/yizaochen/codes/na_mechanics/avg_dcd_noh.py'>